In [93]:
import boto3

# Initialize a session using Amazon EC2
ec2 = boto3.client('ec2')

# Check if the VPC with the tag "MyVPC" already exists
existing_vpcs = ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': ['MyVPC']}])

if existing_vpcs['Vpcs']:
    vpc_id = existing_vpcs['Vpcs'][0]['VpcId']
    print(f"VPC with ID {vpc_id} already exists.")
else:
    # Create VPC
    vpc_response = ec2.create_vpc(CidrBlock='172.16.0.0/16')
    vpc_id = vpc_response['Vpc']['VpcId']

    # Enable DNS support and hostnames
    ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={'Value': True})
    ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={'Value': True})

    # Tag the VPC
    ec2.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': 'MyVPC'}])

    print(f"Created VPC with ID: {vpc_id}")

Created VPC with ID: vpc-0f096f5d17ec9d000


In [96]:
# Create Public Subnets
# Check if the VPC with the tag "MyVPC" already exists
vpc_name='MyVPC'
vpcs = ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': [vpc_name]}])

if vpcs['Vpcs']:
    vpc_id = vpcs['Vpcs'][0]['VpcId']
    print(f"VPC with Tag <{vpc_name}> has the following ID: {vpc_id}")
else:
    # Create VPC
    vpc_response = ec2.create_vpc(CidrBlock='172.16.0.0/16')
    vpc_id = vpc_response['Vpc']['VpcId']

    # Enable DNS support and hostnames
    ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={'Value': True})
    ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={'Value': True})

    # Tag the VPC
    ec2.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': 'MyVPC'}])

    print(f"Created VPC with ID: {vpc_id}")

# Create Public Subnets
public_subnets = [
    {'CidrBlock': '172.16.1.0/24', 'AvailabilityZone': 'eu-west-1a', 'Tag': 'public_subnet_1'},
    {'CidrBlock': '172.16.2.0/24', 'AvailabilityZone': 'eu-west-1b', 'Tag': 'public_subnet_2'},
    {'CidrBlock': '172.16.3.0/24', 'AvailabilityZone': 'eu-west-1c', 'Tag': 'public_subnet_3'}
]

public_subnet_ids = []

for subnet in public_subnets:
    try:
        response = ec2.create_subnet(
            VpcId=vpc_id,
            CidrBlock=subnet['CidrBlock'],
            AvailabilityZone=subnet['AvailabilityZone']
        )
        subnet_id = response['Subnet']['SubnetId']
        public_subnet_ids.append(subnet_id)
        ec2.create_tags(Resources=[subnet_id], Tags=[{'Key': 'Name', 'Value': subnet['Tag']}])
        # Enable MapPublicIpOnLaunch
        ec2.modify_subnet_attribute(SubnetId=subnet_id, MapPublicIpOnLaunch={'Value': True})

        print(f"Created Public Subnet(s) {subnet['Tag']} with ID: {subnet_id}")
    except ClientError as e:
        if 'InvalidSubnet.Conflict' in str(e):
            print(f"Subnet with CIDR block {subnet['CidrBlock']} already exists. Skipping creation.")
        else:
            raise

print(f"The following new public Subnets with these IDs have been created: {public_subnet_ids}")

VPC with Tag <MyVPC> has the following ID: vpc-0f096f5d17ec9d000
Created Public Subnet public_subnet_1 with ID: subnet-017bcae341bd7262e
Created Public Subnet public_subnet_2 with ID: subnet-009aedca27de12145
Created Public Subnet public_subnet_3 with ID: subnet-03f1308c98d75bb0a
VPC ID: vpc-0f096f5d17ec9d000
The following new public Subnets with these IDs have been created: ['subnet-017bcae341bd7262e', 'subnet-009aedca27de12145', 'subnet-03f1308c98d75bb0a']


In [98]:
# Create Private Subnets
# Check if the VPC with the tag "MyVPC" already exists
vpc_name='MyVPC'
vpcs = ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': [vpc_name]}])

if vpcs['Vpcs']:
    vpc_id = vpcs['Vpcs'][0]['VpcId']
    print(f"VPC with Tag <{vpc_name}> has the following ID: {vpc_id}")
else:
    # Create VPC
    vpc_response = ec2.create_vpc(CidrBlock='172.16.0.0/16')
    vpc_id = vpc_response['Vpc']['VpcId']

    # Enable DNS support and hostnames
    ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={'Value': True})
    ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={'Value': True})

    # Tag the VPC
    ec2.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': 'MyVPC'}])

    print(f"Created VPC with ID: {vpc_id}")

# Create Private Subnets
private_subnets = [
    {'CidrBlock': '172.16.4.0/24', 'AvailabilityZone': 'eu-west-1a', 'Tag': 'private_subnet_1'},
    {'CidrBlock': '172.16.5.0/24', 'AvailabilityZone': 'eu-west-1b', 'Tag': 'private_subnet_2'},
    {'CidrBlock': '172.16.6.0/24', 'AvailabilityZone': 'eu-west-1c', 'Tag': 'private_subnet_3'}
]

private_subnet_ids = []

for subnet in private_subnets:
    try:
        response = ec2.create_subnet(
            VpcId=vpc_id,
            CidrBlock=subnet['CidrBlock'],
            AvailabilityZone=subnet['AvailabilityZone']
        )
        subnet_id = response['Subnet']['SubnetId']
        private_subnet_ids.append(subnet_id)
        ec2.create_tags(Resources=[subnet_id], Tags=[{'Key': 'Name', 'Value': subnet['Tag']}])

        print(f"Created Private Subnet(s) {subnet['Tag']} with ID: {subnet_id}")

    except ClientError as e:
        if 'InvalidSubnet.Conflict' in str(e):
            print(f"Subnet with CIDR block {subnet['CidrBlock']} already exists. Skipping creation.")
        else:
            raise

print(f"he following new private Subnets with these IDs have been created: {private_subnet_ids}")

VPC with Tag <MyVPC> has the following ID: vpc-0f096f5d17ec9d000
Subnet with CIDR block 172.16.4.0/24 already exists. Skipping creation.
Subnet with CIDR block 172.16.5.0/24 already exists. Skipping creation.
Subnet with CIDR block 172.16.6.0/24 already exists. Skipping creation.
he following new private Subnets with these IDs have been created: []


In [100]:
# Check if an Internet Gateway is already attached to the VPC
igws = ec2.describe_internet_gateways(Filters=[{'Name': 'attachment.vpc-id', 'Values': [vpc_id]}])

if igws['InternetGateways']:
    igw_id = igws['InternetGateways'][0]['InternetGatewayId']
    print(f"Internet Gateway with ID {igw_id} is already attached to the VPC {vpc_id}.")
else:
    # Attach an Internet Gateway to the VPC
    igw_response = ec2.create_internet_gateway()
    igw_id = igw_response['InternetGateway']['InternetGatewayId']
    ec2.attach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_id)
    print(f"Created and attached Internet Gateway with ID: {igw_id}")

Created and attached Internet Gateway with ID: igw-027a4c42dc4204ceb


In [101]:
# Create a custom route table
route_table_response = ec2.create_route_table(VpcId=vpc_id)
route_table_id = route_table_response['RouteTable']['RouteTableId']
print(f"Created Route Table with ID: {route_table_id}")

# Create a route to the Internet Gateway
ec2.create_route(RouteTableId=route_table_id, DestinationCidrBlock='0.0.0.0/0', GatewayId=igw_id)
print(f"Created route to Internet Gateway in Route Table with ID: {route_table_id}")

# Associate the route table with all of the public subnets
for subnet_id in public_subnet_ids:
    ec2.associate_route_table(RouteTableId=route_table_id, SubnetId=subnet_id)
    print(f"Associated Route Table with ID: {route_table_id} to Subnet with ID: {subnet_id}")

print(f"VPC ID: {vpc_id}")
print(f"Public Subnet IDs: {public_subnet_ids}")

Created Route Table with ID: rtb-0481b33f494773a7b
Created route to Internet Gateway in Route Table with ID: rtb-0481b33f494773a7b
Associated Route Table with ID: rtb-0481b33f494773a7b to Subnet with ID: subnet-017bcae341bd7262e
Associated Route Table with ID: rtb-0481b33f494773a7b to Subnet with ID: subnet-009aedca27de12145
Associated Route Table with ID: rtb-0481b33f494773a7b to Subnet with ID: subnet-03f1308c98d75bb0a
VPC ID: vpc-0f096f5d17ec9d000
Public Subnet IDs: ['subnet-017bcae341bd7262e', 'subnet-009aedca27de12145', 'subnet-03f1308c98d75bb0a']


In [103]:
# Create Security Group

import boto3
from botocore.exceptions import ClientError

ec2 = boto3.client('ec2')

# Retrieve the VPC ID with the tag Name=MyVPC
vpcs = ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': ['MyVPC']}])
vpc_id = vpcs['Vpcs'][0]['VpcId']

# Function to check if a security group already exists
def get_security_group_id(group_name, vpc_id):
    try:
        response = ec2.describe_security_groups(
            Filters=[
                {'Name': 'group-name', 'Values': [group_name]},
                {'Name': 'vpc-id', 'Values': [vpc_id]}
            ]
        )
        return response['SecurityGroups'][0]['GroupId']
    except IndexError:
        return None

# Security Group name
group_name = 'SG-OpenSearch'

# Check if the security group already exists
security_group_id = get_security_group_id(group_name, vpc_id)

if security_group_id:
    print(f"Security Group '{group_name}' already exists with ID: {security_group_id}")
else:
    # Create Security Group
    try:
        response = ec2.create_security_group(
            GroupName=group_name,
            Description='Security group for OpenSearch',
            VpcId=vpc_id
        )
        security_group_id = response['GroupId']

        ec2.create_tags(Resources=[security_group_id], Tags=[{'Key': 'Name', 'Value': 'SG-OpenSearch'}])

        # Define Ingress Rules
        ingress_rules = [
            {
                'IpProtocol': 'tcp',
                'FromPort': 443,
                'ToPort': 443,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 443,
                'ToPort': 443,
                'Ipv6Ranges': [{'CidrIpv6': '::/0'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'Ipv6Ranges': [{'CidrIpv6': '::/0', 'Description': 'SSH ip6'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'Ipv6Ranges': [{'CidrIpv6': '::/0'}]
            }
        ]

        # Authorize Ingress Rules
        for rule in ingress_rules:
            ec2.authorize_security_group_ingress(
                GroupId=security_group_id,
                IpPermissions=[rule]
            )

        # Define Egress Rules
        egress_rules = [
            {
                'IpProtocol': 'tcp',
                'FromPort': 0,
                'ToPort': 65535,
                'Ipv6Ranges': [{'CidrIpv6': '::/0', 'Description': 'all out ipv6'}]
            }
        ]

        # Authorize Egress Rules
        for rule in egress_rules:
            ec2.authorize_security_group_egress(
                GroupId=security_group_id,
                IpPermissions=[rule]
            )

        print(f"Security Group '{group_name}' created with ID: {security_group_id}")

    except ClientError as e:
        print(f"An error occurred: {e}")

print(f"Security Group ID: {security_group_id}")

Security Group 'SG-OpenSearch' created with ID: sg-02df17183f1cd5f68
Security Group ID: sg-02df17183f1cd5f68
